In [1]:
from nn import QLearningAgent, QNetwork
from verify_data import FishGame, ParseError

In [2]:
import os

memories = []
for filename in os.listdir('data'):
    filepath = os.path.join('data', filename)
    with open(filepath, 'r') as f:
        try:
            print(f"{filename}")
            game = FishGame(f.readlines())
            for player in game.players:
                memories += game.memory(player)
        except ParseError as e:
            print(f"{filename}: {e}")
            break

12-10_11:07.txt
12-3_14:05.txt
12-3_11:30.txt
12-3_14:27.txt
12-6_11:08.txt
12-3_11:12.txt
1-15_11:15.txt


In [3]:
# with open('data/1-15_11:15.txt', 'r') as f:
    # game = FishGame(f.readlines())

In [ ]:
agent = QLearningAgent(memories)
agent.train(200)

/Users/Yourui/Documents/Fish/.env/lib/python3.13/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
/Users/Yourui/Documents/Fish/nn.py:64: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  return torch.FloatTensor(x).to(self.device)
/Users/Yourui/Documents/Fish/nn.py:38: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  'ask_card': F.softmax(ask_card),
/Users/Yourui/Documents/Fish/nn.py:85: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting

KeyboardInterrupt: 